In [1]:
a = 2
a

2

In [ ]:
A = [1.0 2.0; 3.0 4.0]
eachcol(A)

In [ ]:
using Yao.BitBasis
using Flux

d = Chain(Dense(nq, 10, relu), Dense(10, 1, sigmoid))
nparams = sum(length, Flux.params(d))
println("Number of parameters in critic: $nparams")
batchsize = 5
function postprocess(g_output::Vector) #turns output of measure  into an Int vector
    result = []
    for i in 1:N
        push!(result, g_output[1][end - i + 1])
    end
    Int.(result)
end

function d_postprocess(measurement::Vector, nbatch = batchsize)
    aa = breflect.(measurement)
    ret = Matrix(undef, nq, nbatch)
    for i in 1:nbatch
        ret[:,i] = [aa[i]...]
    end
    return ret
end
result = measure(reg, nshots = batchsize)
d(d_postprocess(result))

function reinforce_grad_loss(theta, nbatch)
    dispatch!(g, theta)
    T = Float64
    sampled = Dict{BitStr{nq, BigInt}, Vector{T}}()
    measurements = measure(reg, nshots = nbatch)
    discriminator_output = log.(d(d_postprocess(measurements, nbatch)))
    #Initializing temporary matrices and vectors for the optimized version of the algorithm. Note: Do NOT need to reset these temporary matrices at the end of each iteration of the for loop.
    temp_m = Matrix{T}(undef, dim, dim)
    temp_grad_m = Array{T}(undef, nparams, dim, dim)
    probabilities = Vector{T}(undef, nq)
    grad_probabilities = Matrix{T}(undef, nparams, nq)
    grad_p = Matrix{T}(undef, nparams, nbatch)

    for i in 1:nbatch
        cur_bitstr = measurements[i]
        if haskey(sampled, cur_bitstr)
            # println("SAMPLED AGAIN")
            grad_p[:,i] = sampled[cur_bitstr]
        else
            log_grad = log_grad_opt(FLOYao.zero_state(nq), theta, cur_bitstr, temp_m, temp_grad_m, probabilities, grad_probabilities)
            grad_p[:,i] = log_grad
            sampled[cur_bitstr] = log_grad
        end
    end
    return vec(mean(discriminator_output.*grad_p, dims = 2))
end
reinforce_grad_loss(p, 10)